# **Chuyển dữ liệu thô từ file excel thành file json**

In [2]:
import pandas as pd
import json
import os

In [3]:
ROOT_PATH = "D:/Study/PTMXH/Project/SNA2002/Data/Geo"

In [4]:
df = pd.read_excel(os.path.join(ROOT_PATH, "hcm_geoboundaries_raw_data.xlsx"))
df.head()

,id,ward,district,coordinates
0,101,Thị Trấn Tân Túc,Huyện Bình Chánh,"{""value"":[[[106.597351074219,10.7012920379639,..."
1,102,Xã An Phú Tây,Huyện Bình Chánh,"{""value"":[[[106.59790802002,10.7013263702393,1..."
2,103,Xã Bình Chánh,Huyện Bình Chánh,"{""value"":[[[106.569549560547,10.6794986724854,..."
3,104,Xã Bình Hưng,Huyện Bình Chánh,"{""value"":[[[106.69319152832,10.7398862838745,1..."
4,105,Xã Bình Lợi,Huyện Bình Chánh,"{""value"":[[[106.506706237793,10.7932138442993,..."


In [5]:
def split_coord(s):
    """
    Hàm chuyển giá trị trong cột coordinates thành mảng các tọa độ
    """
    coords = eval(s)['value']
    assert len(coords) == 1
    assert len(coords[0]) == 1
    coords = coords[0][0]
    new_coords = []
    for i in range(0, len(coords), 2):
        new_coords.append([coords[i], coords[i+1]])
    return [new_coords]

df.coordinates = df.coordinates.apply(split_coord)
df.head()

,id,ward,district,coordinates
0,101,Thị Trấn Tân Túc,Huyện Bình Chánh,"[[[106.597351074219, 10.7012920379639], [106.5..."
1,102,Xã An Phú Tây,Huyện Bình Chánh,"[[[106.59790802002, 10.7013263702393], [106.59..."
2,103,Xã Bình Chánh,Huyện Bình Chánh,"[[[106.569549560547, 10.6794986724854], [106.5..."
3,104,Xã Bình Hưng,Huyện Bình Chánh,"[[[106.69319152832, 10.7398862838745], [106.69..."
4,105,Xã Bình Lợi,Huyện Bình Chánh,"[[[106.506706237793, 10.7932138442993], [106.5..."


In [6]:
data = {"type":"FeatureCollection", "features":[]}
for i, row in df.iterrows():
    zone = {"type":"Feature", "properties":{}, "geometry":{"type":"MultiPolygon", "coordinates":[]}}
    zone["properties"]["id"] = row.id
    zone["properties"]["name"] = row.ward + ", " + row.district if row.id > 100 else row.district
    zone["geometry"]["coordinates"].append(row.coordinates)
    data["features"].append(zone)

In [7]:
with open(os.path.join(ROOT_PATH, "hcm_geoboundaries.json"), "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False)